## Setting up the environment

In [1]:
!pip install hydra-core==1.1
!pip install scipy==1.7.3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!nvidia-smi

Fri Jul 15 07:47:54 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
BRANCH = 'main'
# If you're using Google Colab and not running locally, uncomment and run this cell.
!apt-get install sox libsndfile1 ffmpeg
!pip install wget unidecode pynini==2.1.4
!python -m pip install git+https://github.com/NVIDIA/NeMo.git@$BRANCH#egg=nemo_toolkit[all]

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libsndfile1 is already the newest version (1.0.28-4ubuntu0.18.04.2).
ffmpeg is already the newest version (7:3.4.11-0ubuntu0.1).
sox is already the newest version (14.4.2-3ubuntu0.18.04.1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/NVIDIA/NeMo.git (to revision main) to /tmp/pip-install-_7mjpoty/nemo-toolkit_4032aab38864486e80d6a98edb94210a
  Running command git clone -q https://github.com/NVIDIA/NeMo.git /tmp/pip-install-_7mjpoty/nemo-toolkit_4032aab38864486e80d6a98edb94210a
  Using cached matplotlib-3.5.2-cp37-cp3

In [4]:
home_path = !(echo $HOME)
home_path = home_path[0]
print(home_path)

/root


In [5]:
!sudo dpkg --configure -a

In [6]:
#=== load the repo and data (Thanks Synthbot) ===
!sudo apt -qq install -y sox
!git clone "https://github.com/synthbot-anon/synthbot.git" /content/synthbot
!(cd /content/synthbot; git checkout experimental)
!python3 -m pip install pysoundfile 
!python3 -m pip install librosa

sox is already the newest version (14.4.2-3ubuntu0.18.04.1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.
fatal: destination path '/content/synthbot' already exists and is not an empty directory.
Already on 'experimental'
Your branch is up to date with 'origin/experimental'.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Preparing the Dataset for Fastpitch finetuning

In [7]:
#downloading the dataset
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=12ClseQAzkqsqCNMClmptvbSjfs2Ulcei' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=12ClseQAzkqsqCNMClmptvbSjfs2Ulcei" -O applejack.tar && rm -rf /tmp/cookies.txt

--2022-07-15 07:49:26--  https://docs.google.com/uc?export=download&confirm=t&id=12ClseQAzkqsqCNMClmptvbSjfs2Ulcei
Resolving docs.google.com (docs.google.com)... 173.194.79.139, 173.194.79.101, 173.194.79.113, ...
Connecting to docs.google.com (docs.google.com)|173.194.79.139|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-0c-84-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/ujuujv7o6laeod06f98jdpiva6qcbg5h/1657871325000/05434657448262868821/*/12ClseQAzkqsqCNMClmptvbSjfs2Ulcei?e=download&uuid=b8887c70-193a-45b4-93d5-1c401fbc5c08 [following]
--2022-07-15 07:49:27--  https://doc-0c-84-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/ujuujv7o6laeod06f98jdpiva6qcbg5h/1657871325000/05434657448262868821/*/12ClseQAzkqsqCNMClmptvbSjfs2Ulcei?e=download&uuid=b8887c70-193a-45b4-93d5-1c401fbc5c08
Resolving doc-0c-84-docs.googleusercontent.com (doc-0c-84-docs.googleusercontent.com)... 142.251.18.132, 

In [8]:
skip_noisy = True # Disable to train with Noisy Data included
allowed_emotions = """
Anxious
Angry
Annoyed
Amused
Confused
Crazy
Disgust
Exhausted
Fear
Happy
Neutral
Sad
Serious
Singing
Shouting
Surprised
Smug
Love
Sarcastic
Tired
Whispering
Whining
""".split("\n")[1:-1]

In [9]:
percentage_training_data = 0.95 # 90% of Data will be used for training, 5% for Validation

In [10]:
import sys
sys.path.append('/content/synthbot/src')
from ponysynth.corpus import ClipperArchive, phoneme_transcription
import librosa
import subprocess

/usr/local/lib/python3.7/dist-packages/resampy/interpn.py:114: NumbaWarning: The TBB threading layer requires TBB version 2019.5 or later i.e., TBB_INTERFACE_VERSION >= 11005. Found TBB_INTERFACE_VERSION = 9107. The TBB threading layer is disabled.
  _resample_loop_p(x, t_out, interp_win, interp_delta, num_table, scale, y)


In [11]:
archive_fn = "/content/applejack.tar"
archive = ClipperArchive(archive_fn)

In [12]:
data_path = 'applejack'
allowed_emotions = [x.lower() for x in allowed_emotions]
!mkdir {data_path}
!mkdir {data_path+"/out"}
all_clips = []; all_clips_arpa = []; skipped_count=0; too_short_count=0; emotion_skip=0
for key in archive.keys(): # write the audio files for processing in bash terminal
  audio = archive.read_audio(key)
  audio_fn = '{}/{}.wav'.format(data_path, key)
  audio_fn_ = '{}/{}.wav'.format(data_path+"/out", key)
  with open(audio_fn, 'wb') as audio_out:
    audio_out.write(audio.read())

In [13]:
%%script bash
# trim all 48Khz files
cd applejack
for input in *.wav; do
  output="out/$input"
  sox "$input" "$output" silence 1 0.05 0.1% reverse silence 1 0.05 0.1% reverse;
done

In [14]:
import os

should_continue = 0 # should run "continue" command inside outer loop.
for key in archive.keys():
  label = archive.read_label(key)
  audio_fn = '{}/{}.wav'.format(data_path, key)
  audio_fn_ = '{}/{}.wav'.format(data_path+"/out", key)
  if (label['noise'] in ['Very Noisy','Noisy']) and skip_noisy: os.remove(audio_fn_); skipped_count+=1; continue

  for tag in label['tags']:
     if tag.lower() not in allowed_emotions:
       try: os.remove(audio_fn_)
       except: pass
       print(tag+" emotion not in list"); emotion_skip+=1; should_continue = 1; break # this is supposed to break the outer loop
  if should_continue: should_continue = 0; continue

  audio = archive.read_audio(key)
  transcript = label['utterance']['content']
  
  if os.stat(audio_fn_).st_size < 71602:
    #print("Skipping Audio, Duration: "+str(len(librosa.core.load(audio_fn_, sr=48000)[0])/48000))
    try: os.remove(audio_fn_)
    except: pass
    too_short_count+=1; continue # Skips files based on size.

  filelist_line = "{}|{}".format(audio_fn_, transcript)
  all_clips.append(filelist_line)
  filelist_line = "{}|{}".format(audio_fn_, phoneme_transcription(label))
  all_clips_arpa.append(filelist_line)
    
print(str(skipped_count)+" Files are too Noisy.")
print(str(emotion_skip)+" Files contain an emotion not in permitted emotions")
print(str(too_short_count)+" Files are too short")
print(str(len(list(archive.keys()))-(skipped_count+too_short_count+emotion_skip))+" Files kept in dataset.")

2768 Files are too Noisy.
0 Files contain an emotion not in permitted emotions
203 Files are too short
2486 Files kept in dataset.


In [15]:
# shuffle the training data
import random
random.seed(0)
random.shuffle(all_clips)

# get train, test, validation splits
num_clips = len(all_clips)
train_end = int(num_clips * percentage_training_data)

train = all_clips[:train_end]
validation = all_clips[train_end:]

train_arpa = all_clips_arpa[:train_end]
validation_arpa = all_clips_arpa[train_end:]

# dump the info to filelist files
with open('train_filelist.txt', 'w') as train_out:
  train_out.write('\n'.join(train)+'\n'+'\n'.join(train_arpa)+"")
with open('val_filelist.txt', 'w') as val_out:
  val_out.write('\n'.join(validation)+'\n'+'\n'.join(validation_arpa)+"")

In [16]:
!jupyter nbextension enable --py --sys-prefix widgetsnbextension

Enabling notebook extension jupyter-js-widgets/extension...
Paths used for configuration of notebook: 
    	/usr/etc/jupyter/nbconfig/notebook.json
      - Validating: OK
Paths used for configuration of notebook: 
    	/usr/etc/jupyter/nbconfig/notebook.json


In [17]:
#imports
import os
import json
import random
import librosa

import torch
import IPython.display as ipd
from matplotlib.pyplot import imshow
from matplotlib import pyplot as plt
from pathlib import Path

from nemo.collections.tts.models import FastPitchModel

[NeMo W 2022-07-15 07:50:40 optimizers:55] Apex was not found. Using the lamb or fused_adam optimizer will error out.
[NeMo W 2022-07-15 07:50:40 experimental:28] Module <class 'nemo.collections.tts.torch.tts_tokenizers.IPATokenizer'> is experimental, not ready for production and is not fully supported. Use at your own risk.


In [18]:
#creating the json files for training and validation
with open('/content/train_filelist.txt') as f :
  records = f.readlines()

print("Number of records : ",len(records))

train_manifest = 'fastpitch_train.json'

train_rec = []
random.shuffle(records)
count = 0

for i in records :
  #if count > 1200 :
  #  break
  
  i = i.split('|')
  audio_filepath = i[0]
  text = i[-1].strip('\n')
  if '{' in text :
    #print(text)
    continue
  count = count + 1

  duration = librosa.get_duration(filename=audio_filepath)
  r = {
       "audio_filepath" : audio_filepath,
       "text" : text,
       "duration" : round(duration,1),
       "text_no_preprocessing" : text
    }

  train_rec.append(r)

with open(train_manifest, "w") as f:
    for s in train_rec:
        f.write(json.dumps(s) + '\n')
        
print("Training Data : ", len(train_rec))

Number of records :  4722
Training Data :  2361


In [19]:
with open('/content/val_filelist.txt') as f :
  records_val = f.readlines()

print("Number of records : ", len(records_val))

val_manifest = 'fastpitch_val.json'
count = 0
val_rec = []
random.shuffle(records_val)

for i in records_val:
  #if count > 50 :
  #  break
    
  i = i.split('|')
  audio_filepath = i[0]
  text = i[-1].strip('\n')

  if '{' in text :
    #print(text)
    continue
  count = count + 1
  duration = librosa.get_duration(filename=audio_filepath)
  r = {
       "audio_filepath" : audio_filepath,
       "text" : text,
       "duration" : round(duration,1),
       "text_no_preprocessing" : text
    }

  val_rec.append(r)

print(len(val_rec))
with open(val_manifest, "w") as f:
    for s in val_rec:
        f.write(json.dumps(s) + '\n')

Number of records :  250
125


In [20]:
FastPitchModel.from_pretrained("tts_en_fastpitch")
nemo_files = [p for p in Path(f"{home_path}/.cache/torch/NeMo/").glob("**/tts_en_fastpitch_align.nemo")]
print(f"Copying {nemo_files[0]} to ./")
Path("./tts_en_fastpitch_align.nemo").write_bytes(nemo_files[0].read_bytes())

[NeMo I 2022-07-15 07:50:41 cloud:66] Downloading from: https://api.ngc.nvidia.com/v2/models/nvidia/nemo/tts_en_fastpitch/versions/1.8.1/files/tts_en_fastpitch_align.nemo to /root/.cache/torch/NeMo/NeMo_1.11.0rc0/tts_en_fastpitch_align/26d7e09971f1d611e24df90c7a9d9b38/tts_en_fastpitch_align.nemo
[NeMo I 2022-07-15 07:50:55 common:789] Instantiating model from pre-trained checkpoint
[NeMo I 2022-07-15 07:50:58 tokenize_and_classify:87] Creating ClassifyFst grammars.


[NeMo W 2022-07-15 07:51:19 experimental:28] Module <class 'nemo.collections.tts.torch.g2ps.IPAG2P'> is experimental, not ready for production and is not fully supported. Use at your own risk.
[NeMo W 2022-07-15 07:51:20 g2ps:87] apply_to_oov_word=None, This means that some of words will remain unchanged if they are not handled by any of the rules in self.parse_one_word(). This may be intended if phonemes and chars are both valid inputs, otherwise, you may see unexpected deletions in your input.
[NeMo W 2022-07-15 07:51:20 modelPT:150] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    dataset:
      _target_: nemo.collections.tts.torch.data.TTSDataset
      manifest_filepath: /ws/LJSpeech/nvidia_ljspeech_train_clean_ngc.json
      sample_rate: 22050
      sup_data_path: /raid/LJSpeech/supplementary
      sup_data_types:
      - align_prior_matri

[NeMo I 2022-07-15 07:51:20 features:200] PADDING: 1
[NeMo I 2022-07-15 07:51:25 save_restore_connector:243] Model FastPitchModel was successfully restored from /root/.cache/torch/NeMo/NeMo_1.11.0rc0/tts_en_fastpitch_align/26d7e09971f1d611e24df90c7a9d9b38/tts_en_fastpitch_align.nemo.
Copying /root/.cache/torch/NeMo/NeMo_1.11.0rc0/tts_en_fastpitch_align/26d7e09971f1d611e24df90c7a9d9b38/tts_en_fastpitch_align.nemo to ./


187023360

In [21]:
!wget https://raw.githubusercontent.com/nvidia/NeMo/main/examples/tts/fastpitch_finetune.py

!mkdir -p conf \
&& cd conf \
&& wget https://raw.githubusercontent.com/nvidia/NeMo/main/examples/tts/conf/fastpitch_align_v1.05.yaml \
&& cd ..

--2022-07-15 07:51:25--  https://raw.githubusercontent.com/nvidia/NeMo/main/examples/tts/fastpitch_finetune.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1751 (1.7K) [text/plain]
Saving to: ‘fastpitch_finetune.py’

fastpitch_finetune. 100%[===================>]   1.71K  --.-KB/s    in 0s      

2022-07-15 07:51:25 (37.6 MB/s) - ‘fastpitch_finetune.py’ saved [1751/1751]

--2022-07-15 07:51:25--  https://raw.githubusercontent.com/nvidia/NeMo/main/examples/tts/conf/fastpitch_align_v1.05.yaml
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 

In [22]:
# additional files
!mkdir -p tts_dataset_files && cd tts_dataset_files \
&& wget https://raw.githubusercontent.com/NVIDIA/NeMo/main/scripts/tts_dataset_files/cmudict-0.7b_nv22.07 \
&& wget https://raw.githubusercontent.com/NVIDIA/NeMo/main/scripts/tts_dataset_files/heteronyms-030921 \
&& wget wget https://raw.githubusercontent.com/NVIDIA/NeMo/$BRANCH/nemo_text_processing/text_normalization/en/data/whitelist/lj_speech.tsv \
&& cd ..

--2022-07-15 07:51:26--  https://raw.githubusercontent.com/NVIDIA/NeMo/main/scripts/tts_dataset_files/cmudict-0.7b_nv22.07
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3722012 (3.5M) [text/plain]
Saving to: ‘cmudict-0.7b_nv22.07’

cmudict-0.7b_nv22.0 100%[===================>]   3.55M  --.-KB/s    in 0.02s   

2022-07-15 07:51:26 (145 MB/s) - ‘cmudict-0.7b_nv22.07’ saved [3722012/3722012]

--2022-07-15 07:51:26--  https://raw.githubusercontent.com/NVIDIA/NeMo/main/scripts/tts_dataset_files/heteronyms-030921
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting

In [29]:
from nemo.collections.tts.torch.g2ps import EnglishG2p
from nemo.collections.tts.torch.data import TTSDataset
from nemo_text_processing.text_normalization.normalize import Normalizer
from nemo.collections.tts.torch.tts_tokenizers import EnglishPhonemesTokenizer, EnglishCharsTokenizer

import torch
from tqdm.notebook import tqdm

In [30]:
# Text normalizer
text_normalizer = Normalizer(
    lang="en", 
    input_case="cased", 
    whitelist="./tts_dataset_files/lj_speech.tsv"
)

text_normalizer_call_kwargs = {
    "punct_pre_process": True,
    "punct_post_process": True
}

# Text tokenizer
text_tokenizer = EnglishCharsTokenizer()

[NeMo I 2022-07-15 07:55:55 tokenize_and_classify:87] Creating ClassifyFst grammars.


In [28]:
def pre_calculate_supplementary_data(sup_data_path, sup_data_types, text_tokenizer, text_normalizer, text_normalizer_call_kwargs):
    # init train and val dataloaders
    stages = ["train", "val"]
    stage2dl = {}
    for stage in stages:
        ds = TTSDataset(
            manifest_filepath=f"fastpitch_{stage}.json",
            sample_rate=16000,
            sup_data_path=sup_data_path,
            sup_data_types=sup_data_types,
            n_fft=1024,
            win_length=1024,
            hop_length=256,
            window="hann",
            n_mels=80,
            lowfreq=0,
            highfreq=8000,
            text_tokenizer=text_tokenizer,
            text_normalizer=text_normalizer,
            text_normalizer_call_kwargs=text_normalizer_call_kwargs

        ) 
        stage2dl[stage] = torch.utils.data.DataLoader(ds, batch_size=1, collate_fn=ds._collate_fn, num_workers=0)

    # iteration over dataloaders
    pitch_mean, pitch_std, pitch_min, pitch_max = None, None, None, None
    for stage, dl in stage2dl.items():
        pitch_list = []
        print(stage)
        print(dl)
        for batch in tqdm(dl, total=len(dl)):
            tokens, tokens_lengths, audios, audio_lengths, attn_prior, pitches, pitches_lengths = batch
            pitch = pitches.squeeze(0)
            pitch_list.append(pitch[pitch != 0])

        if stage == "train":
            pitch_tensor = torch.cat(pitch_list)
            pitch_mean, pitch_std = pitch_tensor.mean().item(), pitch_tensor.std().item()
            pitch_min, pitch_max = pitch_tensor.min().item(), pitch_tensor.max().item()
    
    print("Pitch_mean : ",pitch_mean)
    print("Pitch_std : ",pitch_std)
    print("Pitch_min : ",pitch_min)
    print("Pitch_max : ",pitch_max)
            
    return pitch_mean, pitch_std, pitch_min, pitch_max

## Ran into the following error :

DataLoader worker (pid 6518) is killed by signal: Terminated.

set num_workers=0 in the above function to make it work.


Refs ::

* https://github.com/pytorch/pytorch/issues/8976


In [31]:
fastpitch_sup_data_path = "fastpitch_sup_data_folder"
sup_data_types = ["align_prior_matrix", "pitch"]

pitch_mean, pitch_std, pitch_min, pitch_max = pre_calculate_supplementary_data(
    fastpitch_sup_data_path, sup_data_types, text_tokenizer, text_normalizer, text_normalizer_call_kwargs
)

[NeMo I 2022-07-15 07:56:18 data:186] Loading dataset from fastpitch_train.json.


2361it [00:25, 92.17it/s]

[NeMo I 2022-07-15 07:56:44 data:221] Loaded dataset with 2361 files.
[NeMo I 2022-07-15 07:56:44 data:223] Dataset contains 1.55 hours.
[NeMo I 2022-07-15 07:56:44 data:321] Pruned 0 files. Final dataset contains 2361 files
[NeMo I 2022-07-15 07:56:44 data:324] Pruned 0.00 hours. Final dataset contains 1.55 hours.


[NeMo I 2022-07-15 07:56:44 data:186] Loading dataset from fastpitch_val.json.


125it [00:01, 87.08it/s]

...

train


  0%|          | 0/2361 [00:00<?, ?it/s]

val


  0%|          | 0/125 [00:00<?, ?it/s]

Pitch_mean :  324.42596435546875
Pitch_std :  112.51094055175781
Pitch_min :  65.4063949584961
Pitch_max :  2093.004638671875


## finetuning the Fastpitch model

In [32]:
#setting up wandb credentials
#weights and biases 
wandb_api_key = "d06dcb679ec9cf0d71c81cb750d142b6b90f3d25"
wandb_project_name = "applejack"
wandb_run_name = "applejack"

!wandb login ${wandb_api_key}

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
#resuming the training 
#+init_from_ptl_ckpt=Fastpitch_epoch_356.ckpt
! (python fastpitch_finetune.py --config-name=fastpitch_align_v1.05.yaml \
  train_dataset=fastpitch_train.json \
  validation_datasets=fastpitch_val.json \
  sup_data_path=fastpitch_sup_data \
  phoneme_dict_path=tts_dataset_files/cmudict-0.7b_nv22.07 \
  heteronyms_path=tts_dataset_files/heteronyms-030921 \
  whitelist_path=tts_dataset_files/lj_speech.tsv \
  exp_manager.exp_dir=training_logs \
   +init_from_nemo_model=tts_en_fastpitch_align.nemo \
  trainer.max_epochs=500 ~trainer.max_epochs \
  trainer.check_val_every_n_epoch=7 \
  model.train_ds.dataloader_params.batch_size=8 model.validation_ds.dataloader_params.batch_size=8 \
  model.n_speakers=1 model.pitch_mean=324.43 model.pitch_std=112.51 \
  model.pitch_fmin=65.41 model.pitch_fmax=2093 model.optim.lr=2e-4 \
  +exp_manager.create_wandb_logger=true \
  +exp_manager.wandb_logger_kwargs.name=${wandb_run_name} \
  +exp_manager.wandb_logger_kwargs.project=${wandb_project_name} \
  ~model.optim.sched model.optim.name=adam trainer.devices=1 trainer.strategy=null \
  +model.text_tokenizer.add_blank_at=true \
)

[NeMo W 2022-07-15 10:13:12 optimizers:55] Apex was not found. Using the lamb or fused_adam optimizer will error out.
[NeMo W 2022-07-15 10:13:13 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/resampy/interpn.py:114: NumbaWarning: The TBB threading layer requires TBB version 2019.5 or later i.e., TBB_INTERFACE_VERSION >= 11005. Found TBB_INTERFACE_VERSION = 9107. The TBB threading layer is disabled.
      _resample_loop_p(x, t_out, interp_win, interp_delta, num_table, scale, y)
    
[NeMo W 2022-07-15 10:13:22 experimental:28] Module <class 'nemo.collections.tts.torch.tts_tokenizers.IPATokenizer'> is experimental, not ready for production and is not fully supported. Use at your own risk.
Using 16bit native Automatic Mixed Precision (AMP)
[NeMo W 2022-07-15 10:13:22 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/pytorch_lightning/loops/utilities.py:95: PossibleUserWarning: `max_epochs` was not set. Setting it to 1000 epochs. To train without an epoch limit, set `max_

In [ ]:
# TODO(oktai15): remove +model.text_tokenizer.add_blank_at=true when we update FastPitch checkpoint
! (python fastpitch_finetune.py --config-name=fastpitch_align_v1.05.yaml \
  train_dataset=fastpitch_train.json \
  validation_datasets=fastpitch_val.json \
  sup_data_path=fastpitch_sup_data \
  phoneme_dict_path=tts_dataset_files/cmudict-0.7b_nv22.07 \
  heteronyms_path=tts_dataset_files/heteronyms-030921 \
  whitelist_path=tts_dataset_files/lj_speech.tsv \
  exp_manager.exp_dir=training_logs \
  +init_from_nemo_model=tts_en_fastpitch_align.nemo \
  trainer.max_epochs=500 ~trainer.max_epochs \
  trainer.check_val_every_n_epoch=7 \
  model.train_ds.dataloader_params.batch_size=8 model.validation_ds.dataloader_params.batch_size=8 \
  model.n_speakers=1 model.pitch_mean=319.86 model.pitch_std=122.54 \
  model.pitch_fmin=65.41 model.pitch_fmax=2068.96 model.optim.lr=2e-4 \
  ~model.optim.sched model.optim.name=adam trainer.devices=1 trainer.strategy=null \
  +model.text_tokenizer.add_blank_at=true \
)

In [ ]:
%ls /home/deepankaracharyya/Disk2/Twilight_sparkle/training_logs/FastPitch/2022-06-20_17-00-59/checkpoints/

FastPitch.nemo
FastPitch--v_loss=1.8499-epoch=15.ckpt
FastPitch--v_loss=1.8572-epoch=12.ckpt
FastPitch--v_loss=1.8701-epoch=17.ckpt
FastPitch--v_loss=1.8701-epoch=18-last.ckpt


In [ ]:
#imports
import os
import json
import librosa

import torch
import IPython.display as ipd
from matplotlib.pyplot import imshow
from matplotlib import pyplot as plt
from pathlib import Path

from nemo.collections.tts.models import FastPitchModel
from nemo.collections.tts.models import HifiGanModel
from nemo.collections.tts.torch.helpers import BetaBinomialInterpolator
import soundfile as sf
import numpy as np

In [ ]:
#helper functions
def infer(spec_gen_model, vocoder_model, str_input, speaker=None):
    """
    Synthesizes spectrogram and audio from a text string given a spectrogram synthesis and vocoder model.
    
    Args:
        spec_gen_model: Spectrogram generator model (FastPitch in our case)
        vocoder_model: Vocoder model (HiFiGAN in our case)
        str_input: Text input for the synthesis
        speaker: Speaker ID
    
    Returns:
        spectrogram and waveform of the synthesized audio.
    """
    with torch.no_grad():
        parsed = spec_gen_model.parse(str_input)
        if speaker is not None:
            speaker = torch.tensor([speaker]).long().to(device=spec_gen_model.device)
        spectrogram = spec_gen_model.generate_spectrogram(tokens=parsed, speaker=speaker)
        audio = vocoder_model.convert_spectrogram_to_audio(spec=spectrogram)
        
    if spectrogram is not None:
        if isinstance(spectrogram, torch.Tensor):
            spectrogram = spectrogram.to('cpu').numpy()
        if len(spectrogram.shape) == 3:
            spectrogram = spectrogram[0]
    if isinstance(audio, torch.Tensor):
        audio = audio.to('cpu').numpy()
    return spectrogram, audio

def custom_infer(transcript, spec_model, vocoder, download=None):
  spec, audio = infer(spec_model, vocoder, transcript)
  print("\n\n\n\n")
  ipd.display(ipd.Audio(audio, rate=22050))
  %matplotlib inline
  plt.show()

def try_infering_with_finetuned_fastpitch(checkpointpath, sentence):
  spec_model = FastPitchModel.load_from_checkpoint(checkpointpath)
  spec_model.eval().cuda()

  vocoder = HifiGanModel.from_pretrained("tts_hifigan")
  vocoder = vocoder.eval().cuda()

  custom_infer(sentence, spec_model, vocoder)

In [ ]:
checkpointpath = "/content/Fastpitch_epoch_356.ckpt"

In [ ]:
try_infering_with_finetuned_fastpitch(checkpointpath, "Trying out the model for the first time and I just cannot contain my excitement.")

[NeMo I 2022-07-10 10:31:45 tokenize_and_classify:87] Creating ClassifyFst grammars.


[NeMo W 2022-07-10 10:32:10 g2ps:87] apply_to_oov_word=None, This means that some of words will remain unchanged if they are not handled by any of the rules in self.parse_one_word(). This may be intended if phonemes and chars are both valid inputs, otherwise, you may see unexpected deletions in your input.
[NeMo W 2022-07-10 10:32:10 modelPT:150] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    dataset:
      _target_: nemo.collections.tts.torch.data.TTSDataset
      manifest_filepath: fastpitch_train.json
      sample_rate: 22050
      sup_data_path: fastpitch_sup_data
      sup_data_types:
      - align_prior_matrix
      - pitch
      n_fft: 1024
      win_length: 1024
      hop_length: 256
      window: hann
      n_mels: 80
      lowfreq: 0
      highfreq: 8000
      max_duration: null
      min_duration: 0.1
      ignore_file: null
      

[NeMo I 2022-07-10 10:32:10 features:200] PADDING: 1
[NeMo I 2022-07-10 10:32:11 cloud:66] Downloading from: https://api.ngc.nvidia.com/v2/models/nvidia/nemo/tts_hifigan/versions/1.0.0rc1/files/tts_hifigan.nemo to /root/.cache/torch/NeMo/NeMo_1.11.0rc0/tts_hifigan/e6da322f0f7e7dcf3f1900a9229a7e69/tts_hifigan.nemo
[NeMo I 2022-07-10 10:32:23 common:789] Instantiating model from pre-trained checkpoint


[NeMo W 2022-07-10 10:32:27 modelPT:150] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    dataset:
      _target_: nemo.collections.tts.data.datalayers.MelAudioDataset
      manifest_filepath: /home/fkreuk/data/train_finetune.txt
      min_duration: 0.75
      n_segments: 8192
    dataloader_params:
      drop_last: false
      shuffle: true
      batch_size: 64
      num_workers: 4
    
[NeMo W 2022-07-10 10:32:27 modelPT:157] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    dataset:
      _target_: nemo.collections.tts.data.datalayers.MelAudioDataset
      manifest_filepath: /home/fkreuk/data/val_finetune.txt
      min_duration: 3
      n_segments: 66150


[NeMo I 2022-07-10 10:32:27 features:200] PADDING: 0


[NeMo W 2022-07-10 10:32:27 features:178] Using torch_stft is deprecated and has been removed. The values have been forcibly set to False for FilterbankFeatures and AudioToMelSpectrogramPreprocessor. Please set exact_pad to True as needed.


[NeMo I 2022-07-10 10:32:27 features:200] PADDING: 0
[NeMo I 2022-07-10 10:32:28 save_restore_connector:243] Model HifiGanModel was successfully restored from /root/.cache/torch/NeMo/NeMo_1.11.0rc0/tts_hifigan/e6da322f0f7e7dcf3f1900a9229a7e69/tts_hifigan.nemo.







In [ ]:
spec_model = FastPitchModel.load_from_checkpoint(checkpointpath)
spec_model.eval().cuda()

vocoder = HifiGanModel.from_pretrained("tts_hifigan")
vocoder = vocoder.eval().cuda()

custom_infer("Hi Deepankar, how are you doing today ?", spec_model, vocoder)

[NeMo I 2022-07-13 04:59:48 tokenize_and_classify:87] Creating ClassifyFst grammars.


[NeMo W 2022-07-13 05:00:14 g2ps:87] apply_to_oov_word=None, This means that some of words will remain unchanged if they are not handled by any of the rules in self.parse_one_word(). This may be intended if phonemes and chars are both valid inputs, otherwise, you may see unexpected deletions in your input.
[NeMo W 2022-07-13 05:00:14 modelPT:150] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    dataset:
      _target_: nemo.collections.tts.torch.data.TTSDataset
      manifest_filepath: fastpitch_train.json
      sample_rate: 22050
      sup_data_path: fastpitch_sup_data
      sup_data_types:
      - align_prior_matrix
      - pitch
      n_fft: 1024
      win_length: 1024
      hop_length: 256
      window: hann
      n_mels: 80
      lowfreq: 0
      highfreq: 8000
      max_duration: null
      min_duration: 0.1
      ignore_file: null
      

[NeMo I 2022-07-13 05:00:14 features:200] PADDING: 1
[NeMo I 2022-07-13 05:00:15 cloud:56] Found existing object /root/.cache/torch/NeMo/NeMo_1.11.0rc0/tts_hifigan/e6da322f0f7e7dcf3f1900a9229a7e69/tts_hifigan.nemo.
[NeMo I 2022-07-13 05:00:15 cloud:62] Re-using file from: /root/.cache/torch/NeMo/NeMo_1.11.0rc0/tts_hifigan/e6da322f0f7e7dcf3f1900a9229a7e69/tts_hifigan.nemo
[NeMo I 2022-07-13 05:00:15 common:789] Instantiating model from pre-trained checkpoint


[NeMo W 2022-07-13 05:00:18 modelPT:150] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    dataset:
      _target_: nemo.collections.tts.data.datalayers.MelAudioDataset
      manifest_filepath: /home/fkreuk/data/train_finetune.txt
      min_duration: 0.75
      n_segments: 8192
    dataloader_params:
      drop_last: false
      shuffle: true
      batch_size: 64
      num_workers: 4
    
[NeMo W 2022-07-13 05:00:18 modelPT:157] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    dataset:
      _target_: nemo.collections.tts.data.datalayers.MelAudioDataset
      manifest_filepath: /home/fkreuk/data/val_finetune.txt
      min_duration: 3
      n_segments: 66150


[NeMo I 2022-07-13 05:00:18 features:200] PADDING: 0


[NeMo W 2022-07-13 05:00:18 features:178] Using torch_stft is deprecated and has been removed. The values have been forcibly set to False for FilterbankFeatures and AudioToMelSpectrogramPreprocessor. Please set exact_pad to True as needed.


[NeMo I 2022-07-13 05:00:18 features:200] PADDING: 0
[NeMo I 2022-07-13 05:00:20 save_restore_connector:243] Model HifiGanModel was successfully restored from /root/.cache/torch/NeMo/NeMo_1.11.0rc0/tts_hifigan/e6da322f0f7e7dcf3f1900a9229a7e69/tts_hifigan.nemo.







In [ ]:
custom_infer("I am so excited for the day !", spec_model, vocoder)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
! cp /content/training_logs2/FastPitch/2022-07-09_19-40-53/checkpoints/FastPitch--v_loss=1.2314-epoch=356-last.ckpt -t /content/drive/MyDrive/TTS_finetuning/Twilight_sparkle

In [ ]:
! cp /content/drive/MyDrive/TTS_finetuning/Twilight_sparkle/FastPitch--v_loss=1.2314-epoch=356-last.ckpt  -t .

In [ ]:
#! cp /content/training_logs2/FastPitch/2022-07-09_19-40-53/checkpoints/FastPitch--v_loss=1.2314-epoch=356-last.ckpt -t /content/
! mv ./FastPitch--v_loss=1.2314-epoch=356-last.ckpt  ./Fastpitch_epoch_356.ckpt

## Preparing the data for HiFiGAN finetuning

In [ ]:
#helper functions
def load_wav(audio_file, target_sr=None):
    with sf.SoundFile(audio_file, 'r') as f:
        samples = f.read(dtype='float32')
        sample_rate = f.samplerate
        if target_sr is not None and target_sr != sample_rate:
            samples = librosa.core.resample(samples, orig_sr=sample_rate, target_sr=target_sr)
    return samples.transpose()

In [ ]:
spec_model = FastPitchModel.load_from_checkpoint("/content/Fastpitch_epoch_356.ckpt")
spec_model.eval().cuda()

[NeMo I 2022-07-12 10:49:29 tokenize_and_classify:87] Creating ClassifyFst grammars.


[NeMo W 2022-07-12 10:49:55 g2ps:87] apply_to_oov_word=None, This means that some of words will remain unchanged if they are not handled by any of the rules in self.parse_one_word(). This may be intended if phonemes and chars are both valid inputs, otherwise, you may see unexpected deletions in your input.
[NeMo W 2022-07-12 10:49:55 modelPT:150] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    dataset:
      _target_: nemo.collections.tts.torch.data.TTSDataset
      manifest_filepath: fastpitch_train.json
      sample_rate: 22050
      sup_data_path: fastpitch_sup_data
      sup_data_types:
      - align_prior_matrix
      - pitch
      n_fft: 1024
      win_length: 1024
      hop_length: 256
      window: hann
      n_mels: 80
      lowfreq: 0
      highfreq: 8000
      max_duration: null
      min_duration: 0.1
      ignore_file: null
      

[NeMo I 2022-07-12 10:49:55 features:200] PADDING: 1


FastPitchModel(
  (mel_loss): MelLoss()
  (pitch_loss): PitchLoss()
  (duration_loss): DurationLoss()
  (aligner): AlignmentEncoder(
    (softmax): Softmax(dim=3)
    (log_softmax): LogSoftmax(dim=3)
    (key_proj): Sequential(
      (0): ConvNorm(
        (conv): Conv1d(384, 768, kernel_size=(3,), stride=(1,), padding=(1,))
      )
      (1): ReLU()
      (2): ConvNorm(
        (conv): Conv1d(768, 80, kernel_size=(1,), stride=(1,))
      )
    )
    (query_proj): Sequential(
      (0): ConvNorm(
        (conv): Conv1d(80, 160, kernel_size=(3,), stride=(1,), padding=(1,))
      )
      (1): ReLU()
      (2): ConvNorm(
        (conv): Conv1d(160, 80, kernel_size=(1,), stride=(1,))
      )
      (3): ReLU()
      (4): ConvNorm(
        (conv): Conv1d(80, 80, kernel_size=(1,), stride=(1,))
      )
    )
  )
  (forward_sum_loss): ForwardSumLoss(
    (log_softmax): LogSoftmax(dim=3)
    (ctc_loss): CTCLoss()
  )
  (bin_loss): BinLoss()
  (preprocessor): AudioToMelSpectrogramPreprocessor(
  

In [ ]:
#dataset preparation
# Get records from the training manifest
manifest_path = "./fastpitch_train.json"
records = []
with open(manifest_path, "r") as f:
    for i, line in enumerate(f):
        records.append(json.loads(line))

beta_binomial_interpolator = BetaBinomialInterpolator()
spec_model.eval()

device = spec_model.device

save_dir = Path("./train_mels")
save_dir.mkdir(exist_ok=True, parents=True)

# Generate a spectrograms (we need to use ground truth alignment for correct matching between audio and mels)
for i, r in enumerate(records):
    audio = load_wav(r["audio_filepath"])
    audio = torch.from_numpy(audio).unsqueeze(0).to(device)
    audio_len = torch.tensor(audio.shape[1], dtype=torch.long, device=device).unsqueeze(0)
    
        
    with torch.no_grad():
        if "normalized_text" in r:
            text = spec_model.parse(r["normalized_text"], normalize=False)
        else:
            text = spec_model.parse(r['text'])
        
        text_len = torch.tensor(text.shape[-1], dtype=torch.long, device=device).unsqueeze(0)
    
        spect, spect_len = spec_model.preprocessor(input_signal=audio, length=audio_len)

        # Generate attention prior and spectrogram inputs for HiFi-GAN
        attn_prior = torch.from_numpy(
          beta_binomial_interpolator(spect_len.item(), text_len.item())
        ).unsqueeze(0).to(text.device)
            
        spectrogram = spec_model.forward(
          text=text, 
          input_lens=text_len, 
          spec=spect, 
          mel_lens=spect_len, 
          attn_prior=attn_prior,
        )[0]
        
        save_path = save_dir / f"mel_{i}.npy"
        np.save(save_path, spectrogram[0].to('cpu').numpy())
        r["mel_filepath"] = str(save_path)

hifigan_manifest_path = "hifigan_train_ft.json"
with open(hifigan_manifest_path, "w") as f:
    for r in records:
        f.write(json.dumps(r) + '\n')

[NeMo W 2022-07-12 10:49:58 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/numba/np/ufunc/parallel.py:363: NumbaWarning: The TBB threading layer requires TBB version 2019.5 or later i.e., TBB_INTERFACE_VERSION >= 11005. Found TBB_INTERFACE_VERSION = 9107. The TBB threading layer is disabled.
      warnings.warn(problem)
    


In [ ]:
# Get records from the training manifest
manifest_path_validation = "./fastpitch_val.json"
records_val = []
with open(manifest_path_validation, "r") as f:
    for i, line in enumerate(f):
        records_val.append(json.loads(line))

save_dir_val = Path("./dev_mels")
save_dir_val.mkdir(exist_ok=True, parents=True)

# Generate a spectrograms (we need to use ground truth alignment for correct matching between audio and mels)
for i, r in enumerate(records_val):
    audio = load_wav(r["audio_filepath"])
    audio = torch.from_numpy(audio).unsqueeze(0).to(device)
    audio_len = torch.tensor(audio.shape[1], dtype=torch.long, device=device).unsqueeze(0)
    
       
    with torch.no_grad():
        if "normalized_text" in r:
            text = spec_model.parse(r["normalized_text"], normalize=False)
        else:
            text = spec_model.parse(r['text'])
        
        text_len = torch.tensor(text.shape[-1], dtype=torch.long, device=device).unsqueeze(0)
    
        spect, spect_len = spec_model.preprocessor(input_signal=audio, length=audio_len)

        # Generate attention prior and spectrogram inputs for HiFi-GAN
        attn_prior = torch.from_numpy(
          beta_binomial_interpolator(spect_len.item(), text_len.item())
        ).unsqueeze(0).to(text.device)
            
        spectrogram = spec_model.forward(
          text=text, 
          input_lens=text_len, 
          spec=spect, 
          mel_lens=spect_len, 
          attn_prior=attn_prior,
        )[0]
        
        save_path_val = save_dir_val / f"mel_{i}.npy"
        np.save(save_path_val, spectrogram[0].to('cpu').numpy())
        r["mel_filepath"] = str(save_path_val)

hifigan_val_manifest_path = "hifigan_val_ft.json"
with open(hifigan_val_manifest_path, "w") as f:
    for r in records_val:
        f.write(json.dumps(r) + '\n')

In [ ]:
! mv hifigan.zip -t ./conf
! cd conf && unzip hifigan.zip
! cd conf && cd hifigan && wget https://raw.githubusercontent.com/nvidia/NeMo/main/examples/tts/conf/hifigan/hifigan.yaml && cd .. 

Archive:  hifigan.zip
   creating: hifigan/
   creating: hifigan/model/
   creating: hifigan/model/validation_ds/
 extracting: hifigan/model/validation_ds/val_ds.yaml  
 extracting: hifigan/model/validation_ds/val_ds_finetune.yaml  
   creating: hifigan/model/train_ds/
 extracting: hifigan/model/train_ds/train_ds.yaml  
 extracting: hifigan/model/train_ds/train_ds_finetune.yaml  
   creating: hifigan/model/generator/
 extracting: hifigan/model/generator/v1.yaml  
 extracting: hifigan/model/generator/v1_44100.yaml  
 extracting: hifigan/model/generator/v2.yaml  
 extracting: hifigan/model/generator/v3.yaml  
 extracting: hifigan/hifigan.yaml    
 extracting: hifigan/hifigan_44100.yaml  
--2022-07-12 10:52:50--  https://raw.githubusercontent.com/nvidia/NeMo/main/examples/tts/conf/hifigan/hifigan.yaml
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|

In [ ]:
%pwd
HifiGanModel.from_pretrained("tts_hifigan")
nemo_files = [p for p in Path(f"{home_path}/.cache/torch/NeMo/").glob("**/tts_hifigan.nemo")]
print(f"Copying {nemo_files[0]} to ./")
Path("./tts_hifigan.nemo").write_bytes(nemo_files[0].read_bytes())

[NeMo I 2022-07-12 10:52:23 cloud:66] Downloading from: https://api.ngc.nvidia.com/v2/models/nvidia/nemo/tts_hifigan/versions/1.0.0rc1/files/tts_hifigan.nemo to /root/.cache/torch/NeMo/NeMo_1.11.0rc0/tts_hifigan/e6da322f0f7e7dcf3f1900a9229a7e69/tts_hifigan.nemo
[NeMo I 2022-07-12 10:52:44 common:789] Instantiating model from pre-trained checkpoint


[NeMo W 2022-07-12 10:52:47 modelPT:150] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    dataset:
      _target_: nemo.collections.tts.data.datalayers.MelAudioDataset
      manifest_filepath: /home/fkreuk/data/train_finetune.txt
      min_duration: 0.75
      n_segments: 8192
    dataloader_params:
      drop_last: false
      shuffle: true
      batch_size: 64
      num_workers: 4
    
[NeMo W 2022-07-12 10:52:47 modelPT:157] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    dataset:
      _target_: nemo.collections.tts.data.datalayers.MelAudioDataset
      manifest_filepath: /home/fkreuk/data/val_finetune.txt
      min_duration: 3
      n_segments: 66150


[NeMo I 2022-07-12 10:52:47 features:200] PADDING: 0


[NeMo W 2022-07-12 10:52:47 features:178] Using torch_stft is deprecated and has been removed. The values have been forcibly set to False for FilterbankFeatures and AudioToMelSpectrogramPreprocessor. Please set exact_pad to True as needed.


[NeMo I 2022-07-12 10:52:47 features:200] PADDING: 0
[NeMo I 2022-07-12 10:52:49 save_restore_connector:243] Model HifiGanModel was successfully restored from /root/.cache/torch/NeMo/NeMo_1.11.0rc0/tts_hifigan/e6da322f0f7e7dcf3f1900a9229a7e69/tts_hifigan.nemo.
Copying /root/.cache/torch/NeMo/NeMo_1.11.0rc0/tts_hifigan/e6da322f0f7e7dcf3f1900a9229a7e69/tts_hifigan.nemo to ./


315386678

## finetuning the HiFiGAN model

In [ ]:
!(python hifigan_finetune.py \
--config-name=hifigan.yaml \
model.train_ds.dataloader_params.batch_size=16 \
model.max_steps=10000 \
model.optim.lr=0.0002 \
~model.optim.sched \
train_dataset=./hifigan_train_ft.json \
validation_datasets=./hifigan_val_ft.json \
exp_manager.exp_dir=training_logs \
+init_from_nemo_model=tts_hifigan.nemo \
trainer.check_val_every_n_epoch=2 \
)

[NeMo W 2022-06-20 18:40:49 optimizers:55] Apex was not found. Using the lamb or fused_adam optimizer will error out.
[NeMo W 2022-06-20 18:40:49 experimental:28] Module <class 'nemo.collections.tts.torch.tts_tokenizers.IPATokenizer'> is experimental, not ready for production and is not fully supported. Use at your own risk.
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
[NeMo I 2022-06-20 18:40:49 exp_manager:287] Experiments will be logged at training_logs/HifiGan/2022-06-20_18-40-49
[NeMo I 2022-06-20 18:40:49 exp_manager:661] TensorboardLogger has been set up
[NeMo W 2022-06-20 18:40:49 nemo_logging:349] /usr/local/lib/python3.7/site-packages/pytorch_lightning/trainer/trainer.py:2319: LightningDeprecationWarning: `Trainer.weights_save_path` has been deprecated in v1.6 and will be removed in v1.8.
      rank_zero_deprecation("`Trainer.weights_save_path` has been deprecated in v1.6 and w

In [ ]:
!(python hifigan_finetune.py \
--config-name=hifigan.yaml \
model.train_ds.dataloader_params.batch_size=8 \
+model.max_epochs=1000 \
model.optim.lr=0.0002 \
~model.optim.sched \
train_dataset=./hifigan_train_ft.json \
validation_datasets=./hifigan_val_ft.json \
exp_manager.exp_dir=training_logs2 \
+init_from_nemo_model=tts_hifigan.nemo \
trainer.check_val_every_n_epoch=5 \
++exp_manager.create_wandb_logger=true \
+exp_manager.wandb_logger_kwargs.name=hifigan02 \
+exp_manager.wandb_logger_kwargs.project=MLP \
)

[NeMo W 2022-07-12 10:55:27 optimizers:55] Apex was not found. Using the lamb or fused_adam optimizer will error out.
[NeMo W 2022-07-12 10:55:27 experimental:28] Module <class 'nemo.collections.tts.torch.tts_tokenizers.IPATokenizer'> is experimental, not ready for production and is not fully supported. Use at your own risk.
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
[NeMo I 2022-07-12 10:55:28 exp_manager:287] Experiments will be logged at training_logs2/HifiGan/2022-07-12_10-55-28
[NeMo I 2022-07-12 10:55:28 exp_manager:661] TensorboardLogger has been set up
wandb: Currently logged in as: convaitech. Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.12.21
wandb: Run data is saved locally in training_logs2/wandb/run-20220712_105530-2022-07-12_10-55-28
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run hifigan02
wandb: ⭐️ View project

In [ ]:
!(python hifigan_finetune.py \
--config-name=hifigan.yaml \
model.train_ds.dataloader_params.batch_size=16 \
+model.max_epochs=1000 \
model.optim.lr=0.0002 \
~model.optim.sched \
train_dataset=./hifigan_train_ft.json \
validation_datasets=./hifigan_val_ft.json \
exp_manager.exp_dir=training_logs2 \
+init_from_ptl_ckpt=hifigan_epoch_24.ckpt \
trainer.check_val_every_n_epoch=5 \
++exp_manager.create_wandb_logger=true \
+exp_manager.wandb_logger_kwargs.name=${wandb_run_name} \
+exp_manager.wandb_logger_kwargs.project=${wandb_project_name} \
)

[NeMo W 2022-07-11 11:02:58 optimizers:55] Apex was not found. Using the lamb or fused_adam optimizer will error out.
[NeMo W 2022-07-11 11:02:59 experimental:28] Module <class 'nemo.collections.tts.torch.tts_tokenizers.IPATokenizer'> is experimental, not ready for production and is not fully supported. Use at your own risk.
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
[NeMo I 2022-07-11 11:02:59 exp_manager:287] Experiments will be logged at training_logs2/HifiGan/2022-07-11_11-02-59
[NeMo I 2022-07-11 11:02:59 exp_manager:661] TensorboardLogger has been set up
wandb: Currently logged in as: convaitech. Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.12.21
wandb: Run data is saved locally in training_logs2/wandb/run-20220711_110301-2022-07-11_11-02-59
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run 2022-07-11_11-02-59
wandb: ⭐️ Vi

In [ ]:
!cp /content/training_logs2/HifiGan/2022-07-12_10-55-28/checkpoints/HifiGan--val_loss=0.3945-epoch=164-last.ckpt -t /content/drive/MyDrive/TTS_finetuning/Twilight_sparkle

## Copy the final weights to the gdrive for future reference

In [ ]:
checkpoint_fp_path = "/content/Fastpitch_epoch_356.ckpt"
checkpoint_hf_path = '/content/training_logs2/HifiGan/2022-07-12_10-55-28/checkpoints/HifiGan--val_loss=0.3945-epoch=164-last.ckpt'

In [ ]:
!cp /content/training_logs2/HifiGan/2022-07-11_07-28-01/checkpoints/HifiGan--val_loss=0.4430-epoch=29-last.ckpt -t /content/drive/MyDrive/TTS_finetuning/Twilight_sparkle

In [ ]:
!cp /content/training_logs2/HifiGan/2022-07-11_07-28-01/checkpoints/HifiGan--val_loss=0.4227-epoch=24.ckpt .
!mv HifiGan--val_loss=0.4227-epoch=24.ckpt hifigan_epoch_24.ckpt

## Load the checkpoints and do the inference

In [ ]:
#imports
import os
import json
import librosa

import torch
import IPython.display as ipd
from matplotlib.pyplot import imshow
from matplotlib import pyplot as plt
from pathlib import Path

from nemo.collections.tts.models import FastPitchModel
from nemo.collections.tts.models import HifiGanModel
from nemo.collections.tts.torch.helpers import BetaBinomialInterpolator
import soundfile as sf
import numpy as np

In [ ]:
spec_model = FastPitchModel.load_from_checkpoint(checkpoint_fp_path)
spec_model.eval().cuda()

vocoder = HifiGanModel.load_from_checkpoint(checkpoint_hf_path)
vocoder = vocoder.eval().cuda()

[NeMo I 2022-07-13 04:49:37 tokenize_and_classify:87] Creating ClassifyFst grammars.


[NeMo W 2022-07-13 04:50:03 g2ps:87] apply_to_oov_word=None, This means that some of words will remain unchanged if they are not handled by any of the rules in self.parse_one_word(). This may be intended if phonemes and chars are both valid inputs, otherwise, you may see unexpected deletions in your input.
[NeMo W 2022-07-13 04:50:03 modelPT:150] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    dataset:
      _target_: nemo.collections.tts.torch.data.TTSDataset
      manifest_filepath: fastpitch_train.json
      sample_rate: 22050
      sup_data_path: fastpitch_sup_data
      sup_data_types:
      - align_prior_matrix
      - pitch
      n_fft: 1024
      win_length: 1024
      hop_length: 256
      window: hann
      n_mels: 80
      lowfreq: 0
      highfreq: 8000
      max_duration: null
      min_duration: 0.1
      ignore_file: null
      

[NeMo I 2022-07-13 04:50:03 features:200] PADDING: 1


[NeMo W 2022-07-13 04:50:04 modelPT:150] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    dataset:
      _target_: nemo.collections.tts.torch.data.VocoderDataset
      manifest_filepath: ./hifigan_train_ft.json
      sample_rate: 22050
      n_segments: 8192
      max_duration: null
      min_duration: 0.75
    dataloader_params:
      drop_last: false
      shuffle: true
      batch_size: 8
      num_workers: 4
    
[NeMo W 2022-07-13 04:50:04 modelPT:157] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    dataset:
      _target_: nemo.collections.tts.torch.data.VocoderDataset
      manifest_filepath: ./hifigan_val_ft.json
      sample_rate: 22050
      n_se

[NeMo I 2022-07-13 04:50:04 features:200] PADDING: 0
[NeMo I 2022-07-13 04:50:04 features:208] STFT using exact pad
[NeMo I 2022-07-13 04:50:04 features:200] PADDING: 0
[NeMo I 2022-07-13 04:50:04 features:208] STFT using exact pad


In [ ]:
custom_infer("Is my voice approaching the threshold it is expected to reach?", spec_model, vocoder)

In [ ]:
custom_infer("Ouch! I got my hand broken!", spec_model, vocoder)

In [ ]:
custom_infer("Trying out the model for the first time and I cannot contain my excitement!", spec_model, vocoder)